In [ ]:
import os
import omegaconf
from shell.utils.experiment_utils import *
from shell.utils.metric import *
import matplotlib.pyplot as plt
from shell.fleet.network import TopologyGenerator

In [ ]:
save_root_dir = "full_data_results"
dataset = "mnist"
algo = "monolithic"
seed = 0

In [ ]:
experiment = os.path.join(save_root_dir, f"{dataset}_{algo}", dataset,algo, f"seed_{seed}")

In [ ]:
config_path = os.path.join(experiment, "hydra_out", ".hydra", "config.yaml")
# read the config file
cfg = omegaconf.OmegaConf.load(config_path)
cfg

In [ ]:
graph, datasets, NetCls, LearnerCls, net_cfg, agent_cfg, train_cfg = setup_experiment(cfg)
len(datasets)

In [ ]:
classes_sequence_list = [dataset.class_sequence for dataset in datasets]
classes_sequence_list

In [ ]:
df_sim = task_similarity(classes_sequence_list, cfg.dataset.num_tasks, cfg.dataset.num_classes_per_task)

In [ ]:
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 2)
df_sim.groupby("task_id").mean()['similarity'].plot(legend=True, label="similarity")
plt.subplot(1, 2, 1)
df_sim.groupby("task_id").mean()['running_similarity'].plot(legend=True, label="running_similarity");

In [ ]:
import networkx as nx
nx.draw(graph)

In [ ]:
# from shell.fleet.fleet import Fleet
# from shell.fleet.gradient_fleet import ModelSyncAgent
# num_coms_per_round = 1
# AgentCls = ModelSyncAgent

# fleet = Fleet(graph, cfg.seed, datasets, num_coms_per_round, AgentCls, NetCls=NetCls,
#                 LearnerCls=LearnerCls, net_kwargs=net_cfg, agent_kwargs=agent_cfg,
#                 train_kwargs=train_cfg)
# # for task_id in range(cfg.dataset.num_tasks):
# for task_id in range(2): 
#     fleet.communicate(task_id)

In [ ]:
from shell.fleet.fleet import Fleet
from shell.fleet.monograd import ModelSyncAgent
num_coms_per_round = 1
AgentCls = ModelSyncAgent

fleet = Fleet(graph, cfg.seed, datasets, num_coms_per_round, AgentCls, NetCls=NetCls,
                LearnerCls=LearnerCls, net_kwargs=net_cfg, agent_kwargs=agent_cfg,
                train_kwargs=train_cfg)
# for task_id in range(cfg.dataset.num_tasks):
for task_id in range(2): 
    fleet.communicate(task_id)